In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [26]:
from inspect import signature
from functools import wraps


def typeassert(*ty_args, **ty_kwargs):
    def decorator(func):
        if not __debug__:
            return func

        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments

        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            bound_values.apply_defaults()
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        msg = 'argument {} must be {}'
                        raise TypeError(
                            msg.format(name, bound_types[name].__name__)
                        )

            return func(*args, **kwargs)
        return wrapper
    return decorator


"""
Basic

    >>> @typeassert(int, int)
    ... def add(x, y):
    ...     return x + y
    >>> add(2, 3)
    5
    >>> add(2, 'hello')
    Traceback (most recent call last):
        ...
    TypeError: argument y must be int

With only specific fields

    >>> @typeassert(int, z=int)
    ... def spam(x, y, z=42):
    ...     print(x, y, z)
    >>> spam(1, 2, 3)
    1 2 3
    >>> spam(1, 'hello', 3)
    1 hello 3
    >>> spam(1, 'hello', 'world')
    Traceback (most recent call last):
        ...
    TypeError: argument z must be int

Inspecting function signature

    >>> spam = spam.__wrapped__
    >>> sig = signature(spam)
    >>> print(sig)
    (x, y, z=42)
    >>> sig.parameters  # doctest: +ELLIPSIS
    mappingproxy(OrderedDict([('x', <Parameter "x">), ('y', <Parameter "y">), ('z', <Parameter "z=42">)]))
    >>> sig.parameters['z'].name
    'z'
    >>> sig.parameters['z'].default
    42
    >>> sig.parameters['z'].kind
    <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>
    >>> bound_types = sig.bind_partial(int, z=int)
    >>> bound_types
    <BoundArguments (x=<class 'int'>, z=<class 'int'>)>
    >>> bound_types.arguments
    {'x': <class 'int'>, 'z': <class 'int'>}
    >>> bound_values = sig.bind(1, 2)
    >>> bound_values.apply_defaults()
    >>> bound_values
    <BoundArguments (x=1, y=2, z=42)>
    >>> bound_values.arguments
    {'x': 1, 'y': 2, 'z': 42}

Edge-case

    >>> @typeassert(int, list)
    ... def bar(x, items=None):
    ...     if items is None:
    ...         items = []
    ...     items.append(x)
    ...     return items
    >>> bar(2)
    Traceback (most recent call last):
        ...
    TypeError: argument items must be list
    >>> bar(2, 3)
    Traceback (most recent call last):
        ...
    TypeError: argument items must be list
    >>> bar(4, [1, 2, 3])
    [1, 2, 3, 4]

"""  # noqa: E501

doctest.testmod()

TestResults(failed=0, attempted=25)